# Problem 1

In [1]:
import numpy as np
import pandas as pd

## Guess the time of each line

In [2]:
rs = np.random.RandomState(1126)
%time df = pd.DataFrame(rs.randint(200, size=(500000, 200)), dtype=np.float64)
%time new_s = df.loc[:, 0].astype(np.int16)
%time df.loc[:, 0] = new_s

CPU times: user 1.31 s, sys: 404 ms, total: 1.72 s
Wall time: 1.81 s
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 11.3 ms
CPU times: user 1.1 s, sys: 217 ms, total: 1.32 s
Wall time: 1.34 s


- The assignment is supprisingly slow! It's almost the same as creating a new `DataFrame` from scratch.

## Let's run again

In [3]:
%time df = pd.DataFrame(rs.randint(200, size=(500000, 200)), dtype=np.float64)
%time new_s = df.loc[:, 0].astype(np.int16)
%time df.loc[:, 0] = new_s

CPU times: user 1.22 s, sys: 435 ms, total: 1.66 s
Wall time: 1.68 s
CPU times: user 5.26 ms, sys: 4.37 ms, total: 9.63 ms
Wall time: 9.07 ms
CPU times: user 1.09 s, sys: 226 ms, total: 1.32 s
Wall time: 1.34 s


- the result is still similar

## Tracing the issue

- `pandas.core.internals`: 6000 lines of code in one file
- the magical `BlockManager` in `pandas.core.internals` mainly controls how `pandas` deals with memory
- `df._data` is the `BlockManager` of `df`
- we can observe the behavior by watching `df._data.blocks`

In [4]:
df = pd.DataFrame(rs.randint(200, size=(500000, 200)), dtype=np.float64)
df._data.blocks

(FloatBlock: slice(0, 200, 1), 200 x 500000, dtype: float64,)

In [5]:
df.loc[:, 0] = df.loc[:, 0].astype(np.int16)
df._data.blocks

(FloatBlock: slice(1, 200, 1), 199 x 500000, dtype: float64,
 IntBlock: slice(0, 1, 1), 1 x 500000, dtype: int16)

- after the deletion, a new `IntBlock` is generated, and the original `FloatBlock` shrinks to size 199 x 500000

## So why is this slow?

After some profiling and tracing, we can find that:

- each block maintains an `np.ndarray` (#cols x #rows)
- `BlockManager` calls `np.delete` to delete one row in `FloatBlock`
- when we assign the column, <span style="color:red">98.46% of time</span> is used to do the deletion
- `np.delete` copy the whole `np.ndarray` except the deleted row to a new `np.ndarray`

## Why does pandas do this?

- explained in the [pandas 2.0 design docs](https://pandas-dev.github.io/pandas2/internal-architecture.html#removal-of-blockmanager-new-dataframe-internals)
    - ancient `pandas` history
    - they wanted to rely on `numpy`, and contiguous memory access produces much better performance in `numpy`
    - so they use `BlockManager` to maintain severy contiguous memory blocks for different types
    - `pandas` developers want to replace `BlockManager` using native C/C++ code and design a new algorithm that won't have this kind of magical problems
- currently
    - `pandas` is designed for those fast operations
    - **`pandas` is not designed to frequently change the sizes or types of the blocks**

## Extended observation

Based our previous observation, we can explore more about the magical behavior.


Try your best to guess the result of the following code:

In [6]:
df.loc[:, 3] = df.loc[:, 3].astype(np.int16)
df._data.blocks

(FloatBlock: [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, ...], 198 x 500000, dtype: float64,
 IntBlock: slice(0, 1, 1), 1 x 500000, dtype: int16,
 IntBlock: slice(3, 4, 1), 1 x 500000, dtype: int16)

- supprisingly, `pandas` doesn't merge the new `np.int16` column to the existing `IntBlock`

In [7]:
df.loc[:, 3] = df.loc[:, 3].astype(np.float64)
df._data.blocks

(FloatBlock: [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, ...], 198 x 500000, dtype: float64,
 IntBlock: slice(0, 1, 1), 1 x 500000, dtype: int16,
 FloatBlock: slice(3, 4, 1), 1 x 500000, dtype: float64)

- after changing the type of a column, the next type changing of that column should be very fast because `pandas` won't do any heavy operation on an existing block

# Other problems

- What happens when you append a row to a `DataFrame`?
- What happens when you add a column to a `DataFrame`?
- How about 1-d arrays (`Series` and `Index`)?

# Conclusion

- `pandas` is not designed to frequently change the sizes or types of the blocks
- try to avoid these kinds of operation by generating a lot of small `DataFrame` or `Series`, and concatenate them in the end
    - this tip can also be applied to `numpy`